# Import library

In [1]:
#import basic module
import os
import time
from ximea import xiapi
import cv2
import time
import numpy as np
import csv

c:\Users\kawaw\anaconda3\envs\kawawaki\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\kawaw\anaconda3\envs\kawawaki\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\kawaw\anaconda3\envs\kawawaki\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Rotational Matrix

0. get data 

In [76]:
camera1 = [78.62,130.64,440.5,1.8744,1.573,2.8379]
camera2 = [70.32,136.91,446.94,1.9131,1.53318,2.7971]
camera3 = [44.27,167.62,516.2,1.47901,1.49827,3.02454]
cameras = np.array([camera1,camera2,camera3])
robot1 = [145.2,-35.9,907.4,0.394,1.898,-2.442]
robot2 = [244.4,-10.4,908.8,0.219,1.81,-2.81]
robot3 = [186,77.9,899,0.078,2.015,2.658]
robots = np.array([robot1,robot2,robot3])

1. make coordinate Matrix from position and angle

In [82]:
def pose2matrix(pose):
    H_c = np.zeros((4,4))
    # Create rotation matrices for each axis
    rotation_x = np.array([[1, 0, 0],
                        [0, np.cos(pose[3]), -np.sin(pose[3])],
                        [0, np.sin(pose[3]), np.cos(pose[3])]])

    rotation_y = np.array([[np.cos(pose[4]), 0, np.sin(pose[4])],
                        [0, 1, 0],
                        [-np.sin(pose[4]), 0, np.cos(pose[4])]])

    rotation_z = np.array([[np.cos(pose[5]), -np.sin(pose[5]), 0],
                        [np.sin(pose[5]), np.cos(pose[5]), 0],
                        [0, 0, 1]])

    # Combine the rotation matrices to get the final rotation matrix : Z rotation -> Y rotation -> X rotation
    rotation_matrix = np.dot(rotation_z, np.dot(rotation_y, rotation_x))

    print("Rotation Matrix:")
    print(rotation_matrix)

    H_c[:3,:3]=rotation_matrix
    H_c[:3,3] = np.array(pose[:3])
    H_c[3,3]=1
    print("transformation matrxi=",H_c)
    return H_c
H_c = np.zeros((3,4,4))
for i in range(3):
    H_c[i] = pose2matrix(pose=cameras[i])
H_r = np.zeros((3,4,4))
for j in range(3):
    H_r[j] = pose2matrix(pose=robots[j])

Rotation Matrix:
[[ 2.10282861e-03 -8.21191656e-01  5.70648616e-01]
 [-6.58998927e-04  5.70648616e-01  8.21194084e-01]
 [-9.99997572e-01 -2.10288724e-03  6.58811814e-04]]
transformation matrxi= [[ 2.10282861e-03 -8.21191656e-01  5.70648616e-01  7.86200000e+01]
 [-6.58998927e-04  5.70648616e-01  8.21194084e-01  1.30640000e+02]
 [-9.99997572e-01 -2.10288724e-03  6.58811814e-04  4.40500000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Rotation Matrix:
[[-0.0353979  -0.77265409  0.6338396 ]
 [ 0.01270076  0.63383805  0.7733615 ]
 [-0.99929259  0.03542562 -0.01262324]]
transformation matrxi= [[-3.53979022e-02 -7.72654093e-01  6.33839602e-01  7.03200000e+01]
 [ 1.27007594e-02  6.33838054e-01  7.73361502e-01  1.36910000e+02]
 [-9.99292589e-01  3.54256191e-02 -1.26232437e-02  4.46940000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Rotation Matrix:
[[-0.07196691 -0.99708092  0.02550295]
 [ 0.0084626   0.02495792  0.99965268]
 [-0.99737112  

3. calculate transformation matrix

In [84]:
T = np.zeros((3,4,4))
for i in range(3):
    T[i] = np.dot(H_r[i],np.linalg.inv(H_c[i]))
print("Transformation Matrix (4x4):")
print(T)

Transformation Matrix (4x4):
[[[-7.82162327e-01 -5.71951899e-01 -2.47170224e-01  3.90291882e+02]
  [ 6.19127243e-01 -7.58020922e-01 -2.05148089e-01  1.04819803e+02]
  [-7.00253615e-02 -3.13488926e-01  9.47006411e-01  5.36703263e+02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 [[-7.12430158e-01 -6.70374230e-01 -2.07464843e-01  4.79003362e+02]
  [ 6.97995442e-01 -7.07462219e-01 -1.10903430e-01  8.69427921e+01]
  [-7.24267370e-02 -2.23820464e-01  9.71935578e-01  5.10139421e+02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 [[ 4.77523109e-01 -7.70390740e-01 -4.22456847e-01  5.12065172e+02]
  [ 8.74256920e-01  4.64476326e-01  1.41196958e-01 -1.11544745e+02]
  [ 8.74443752e-02 -4.36760632e-01  8.95317615e-01  5.06175702e+02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]]


4. roation matrix to angle

In [85]:
def rotation_matrix_to_euler_angles(R):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)

    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        x = np.arctan2(-R[1, 1], R[1, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = 0
    return [x, y, z]

def matrix2position(T):
    x = T[0,3]
    y = T[1,3]
    z = T[2,3]
    return [x,y,z]

for i in range(3):
    print(f"{i+1}-th  transformation matrix")
    # Calculate Euler angles
    [pitch, yaw, roll] = rotation_matrix_to_euler_angles(T[i,:3,:3])

    print(f"Pitch: {pitch} degrees")
    print(f"Yaw: {yaw} degrees")
    print(f"Roll: {roll} degrees")

1-th  transformation matrix
Pitch: -0.3196774504944793 degrees
Yaw: 0.07008271700956438 degrees
Roll: 2.472020201145544 degrees
2-th  transformation matrix
Pitch: -0.22633737052670258 degrees
Yaw: 0.07249020758827139 degrees
Roll: 2.3664284310684893 degrees
3-th  transformation matrix
Pitch: -0.45386234554475346 degrees
Yaw: -0.08755620125001007 degrees
Roll: 1.0708717565962165 degrees


# Evaluation of transformation matrix

## convert Camera to Robot coordinates 

In [86]:
evaluationMatrix = np.zeros((3,3,4,4))
rawData = np.zeros((3,3,4,4))
for i in range(3):
    for j in range(3):
        evaluationMatrix[i][j] = np.dot(T[i],H_c[(i+j+1)%3])
        rawData[i][j] = H_r[(i+j+1)%3]

## Calculate RMSE

In [87]:
def rmse(H_target,H_reconstruct):
    angle_target = rotation_matrix_to_euler_angles(H_target[:3,:3])
    angle_reconstruct= rotation_matrix_to_euler_angles(H_reconstruct[:3,:3])
    pos_target = matrix2position(H_target)
    pos_reconstruct = matrix2position(H_reconstruct)
    rmse_angle = 0
    rmse_pos = 0
    names = ["x","y","z"]
    for i in range(3):
        rmse_angle += (angle_target[i]-angle_reconstruct[i])**2
        rmse_pos += (pos_target[i]-pos_reconstruct[i])**2
        print(names[i],f":: angle : {angle_target[i]-angle_reconstruct[i]}, position : {pos_target[i]-pos_reconstruct[i]}")
    rmse_angle = round(rmse_angle**(1/2),2)
    rmse_pos = round(rmse_pos**(1/2),2)
    print(f"rmse :: angle = {rmse_angle} [rad]")
    print(f"rmse :: position = {rmse_pos} [mm]")


In [88]:
for i in range(3):
    for j in range(3):
        rmse(rawData[i][j],evaluationMatrix[i][j])

x :: angle : -0.13684860972728163, position : 97.88596732927519
y :: angle : 0.0852052876087599, position : 36.71270099380343
z :: angle : -0.24383030899940583, position : -3.3143562194459264
rmse :: angle = 0.29 [rad]
rmse :: position = 104.6 [mm]
x :: angle : -5.805899540570703, position : 53.79429120040453
y :: angle : -0.10233626797622697, position : 178.62834486841064
z :: angle : -0.9596526247817074, position : -70.9009359755637
rmse :: angle = 5.89 [rad]
rmse :: position = 199.57 [mm]
x :: angle : 0.0, position : -2.842170943040401e-14
y :: angle : 0.0, position : -2.1316282072803006e-14
z :: angle : 0.0, position : 0.0
rmse :: angle = 0.0 [rad]
rmse :: position = 0.0 [mm]
x :: angle : -5.647233190015326, position : -42.00259796082497
y :: angle : -0.18293190527028802, position : 135.8901176039132
z :: angle : -0.6957511565262953, position : -72.12944819915674
rmse :: angle = 5.69 [rad]
rmse :: position = 159.48 [mm]
x :: angle : 0.09055774204479716, position : -98.8261497001245